In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ['DISPLAY'] = ':1'
os.environ['NAPARI_ASYNC'] = '1'


from SyMBac.simulation import Simulation
from SyMBac.PSF import PSF_generator
from SyMBac.renderer import Renderer
from SyMBac.PSF import Camera
from SyMBac.misc import get_sample_images
real_image = get_sample_images()["E. coli 100x"]
from joblib import Parallel, delayed


In [3]:
my_simulation = Simulation(
    trench_length=25,
    trench_width=1.15,
    cell_max_length=6.65, #6, long cells # 1.65 short cells
    cell_width= 1, #1 long cells # 0.95 short cells
    sim_length = 300,
    pix_mic_conv = 0.065,
    gravity=0,
    phys_iters=15,
    max_length_var = 0.,
    width_var = 0.,
    lysis_p = 0.00,
    save_dir="/tmp/test/",
    resize_amount = 3,
)

In [ ]:
my_simulation.run_simulation(show_window = True)

/home/gh464/Documents/GitHub/SyMBac/SyMBac/simulation.py:148: UserWarning: You are using show_window = True. If you re-run the simulation (even by re-creating the Simulation object), then for reasons which I do not understand, the state of the simulation is not reset. Restart your notebook or interpreter to re-run simulations.
  warnings.warn("You are using show_window = True. If you re-run the simulation (even by re-creating the Simulation object), then for reasons which I do not understand, the state of the simulation is not reset. Restart your notebook or interpreter to re-run simulations.")


In [606]:
my_simulation.cell_timeseries[13][1].t

13

In [590]:
my_simulation.cell_timeseries[11][0].daughter

In [577]:
my_simulation.cell_timeseries[13][1].just_divided

False

In [454]:
from networkx.algorithms.traversal import dfs_edges, bfs_edges
import networkx as nx

In [153]:
# Repressilator deterministic example
import bioscrape
plt.figure()
m = bioscrape.sbmlutil.import_sbml('repressilator_sbml.xml')


/home/gh464/miniconda3/envs/symbac/lib/python3.10/site-packages/bioscrape/sbmlutil.py:56: UserWarning: Compartments, UnitDefintions, Events, and some other SBML model components are not recognized by bioscrape. Refer to the bioscrape wiki for more information.
  warnings.warn('Compartments, UnitDefintions, Events, and some other SBML model components are not recognized by bioscrape. ' +


<Figure size 640x480 with 0 Axes>

In [558]:
from SyMBac.lineage import Lineage
my_lineage = Lineage(my_simulation)
dfs = dfs_edges(my_lineage.temporal_lineage_graph, source=(1,0))

In [559]:
attrs_n = {(1,0): {'concs': np.array([0,0,0,0,0,0])}}

nx.set_node_attributes(my_lineage.temporal_lineage_graph, attrs_n) 

In [560]:
my_lineage.temporal_lineage_graph.nodes[(1,0)]["concs"]

array([0, 0, 0, 0, 0, 0])

In [561]:
for edge in dfs:
    
    current_node = edge[1]
    parent_node = next(my_lineage.temporal_lineage_graph.predecessors(current_node))
    init_condition = my_lineage.temporal_lineage_graph.nodes[parent_node]["concs"]
    
    s = bioscrape.simulator.ModelCSimInterface(m)
    s.py_prep_deterministic_simulation()
    s.py_set_initial_state(init_condition)
    s.py_set_initial_time(0)
    sim = bioscrape.simulator.SSASimulator()
    timepoints = np.linspace(0,1,100)
    result = sim.py_simulate(s,timepoints)
    set_result = result.py_get_result()[-1]
    
    attrs_n = {current_node: {'concs': set_result}}
    
    nx.set_node_attributes(my_lineage.temporal_lineage_graph, attrs_n) 

KeyError: 'concs'

In [563]:
current_node

(57, 293)

In [566]:
list(my_lineage.temporal_lineage_graph.predecessors(current_node))

[(57, 292), (1, 293)]

In [ ]:
parent_node

In [534]:
list(my_lineage.temporal_lineage_graph.successors((1, 298)))

[(1, 299), (57, 298), (64, 298), (68, 298)]

In [539]:
parent_node

(57, 298)

In [541]:
list(my_lineage.temporal_lineage_graph.predecessors((57, 297)))

[(57, 296), (1, 297)]

In [529]:
list(my_lineage.temporal_lineage_graph.predecessors(parent_node))

[(57, 297), (1, 298)]

In [527]:
my_lineage.temporal_lineage_graph.nodes[parent_node]["concs"]

KeyError: 'concs'

In [545]:
my_lineage.temporal_lineage_graph.nodes

NodeView(((1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (2, 13), (1, 14), (2, 14), (1, 15), (2, 15), (1, 16), (2, 16), (1, 17), (2, 17), (1, 18), (2, 18), (1, 19), (2, 19), (1, 20), (2, 20), (1, 21), (2, 21), (1, 22), (2, 22), (1, 23), (2, 23), (1, 24), (2, 24), (1, 25), (2, 25), (1, 26), (2, 26), (1, 27), (2, 27), (3, 27), (1, 28), (2, 28), (3, 28), (1, 29), (2, 29), (3, 29), (1, 30), (2, 30), (3, 30), (1, 31), (2, 31), (3, 31), (1, 32), (2, 32), (3, 32), (4, 32), (1, 33), (2, 33), (3, 33), (4, 33), (1, 34), (2, 34), (3, 34), (4, 34), (1, 35), (2, 35), (3, 35), (4, 35), (1, 36), (2, 36), (3, 36), (4, 36), (1, 37), (2, 37), (3, 37), (4, 37), (1, 38), (2, 38), (3, 38), (4, 38), (1, 39), (2, 39), (3, 39), (4, 39), (1, 40), (2, 40), (3, 40), (4, 40), (1, 41), (2, 41), (3, 41), (4, 41), (1, 42), (2, 42), (3, 42), (4, 42), (1, 43), (2, 43), (3, 43), (4, 43), (1, 44), (2, 44), (3, 44), (4, 44), (1, 45), (2, 45), (3, 45), (

In [505]:
parent_node

(1, 1)

In [485]:
%%timeit
s = bioscrape.simulator.ModelCSimInterface(m)
s.py_prep_deterministic_simulation()
s.py_set_initial_state(np.array([0,0,0,0,0,0]))
s.py_set_initial_time(0)

sim = bioscrape.simulator.SSASimulator()
timepoints = np.linspace(0,10,100)
result = sim.py_simulate(s,timepoints)
set_result = result.py_get_result()[-1]

1.29 ms ± 4.64 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


array([222., 335., 378.,   0.,   1.,   0.])